In [31]:
# The index convenction is arbitrary, 
# orientation is not important here

vertices = [{0},{1},{2},{3}]

edges = [
        {0,1},
        {0,2},
        {0,3},
        {1,2},
        {1,3},
        {2,3}
]

faces = [
        {1,2,3},
        {0,3,2},
        {0,1,3},
        {0,2,1}
    ]

s = [vertices,edges,faces]
len(s[0])


4

In [43]:
# Generates all possible tuples, note that some might not be valid
def all_tuples(s):
    return [[i,j,k] for i in range(len(s[0])) for j in range(len(s[1])) for k in range(len(s[2]))]

# Check if a tuple is valid, i.e. if the simplex of dimension d-1 is a face of a simplex of dimension d
def valid_tuple(t,s):
    return s[0][t[0]].issubset(s[1][t[1]]) and s[1][t[1]].issubset(s[2][t[2]]) 

# valid = [t for t in all_tuples(s) if valid_tuple(t,s)]


In [42]:
# Enumerates all valid tuples similar to t, but with a different value in the slot d
# There must be 2 of them, and this function returns the one that is different than t
def switch(t,d,s):
    candidates = []
    for i in range(len(s[d])):
        t2 = t.copy()
        t2[d] = i
        candidates.append(t2)
    valid = [x for x in candidates if valid_tuple(x,s)]
    assert(len(valid) == 2)
    valid.remove(t)
    assert(len(valid) == 1)
    return valid[0]

[0, 0, 3]

In [71]:
def list_to_array(list_of_lists,name):
    # Get the number of rows and columns in the list of lists
    rows = len(list_of_lists)
    cols = len(list_of_lists[0])

    # Create the C++ code for the static array of arrays
    cplusplus_code = "static int " + name +"[" + str(rows) + "][" + str(cols) + "] = {\n"

    # Loop through each row in the list of lists
    for row in list_of_lists:
        # Add the opening brace for the row
        cplusplus_code += "  {"

        # Loop through each element in the row
        for element in row:
            # Add the element to the row
            cplusplus_code += str(element) + ", "

        # Remove the trailing comma and space from the row
        cplusplus_code = cplusplus_code[:-2]

        # Add the closing brace for the row
        cplusplus_code += "},\n"

    # Remove the trailing comma and newline from the C++ code
    cplusplus_code = cplusplus_code[:-2] + "\n};\n"

    return cplusplus_code

In [72]:
# Builds a table for the switch operation of dimension d
def table(d,name,s):
    # Generates a table for switch_vertex
    sv = all_tuples(s)

    for i in range(len(sv)):
        if not valid_tuple(sv[i],s):
            sv[i] = [-1,-1,-1]
        else:
            sv[i] = switch(sv[i],d,s)

    return list_to_array(sv,name)

In [73]:
vertex = table(0,"vertex",s)
edge = table(1,"edge",s)
face = table(2,"face",s)

In [74]:
text_file = open("tuple_tables.cpp", "w")
text_file.write('// Autogenerate by 3d_switch_tables.ipynb\n')
text_file.write(vertex)
text_file.write(edge)
text_file.write(face)
text_file.write('Footer\n')

text_file.close()